In [68]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [127]:
import calendar

In [4]:
import pandas as pd
import csv, json, os, math, numbers, pandas, re, scipy, scipy.sparse, shutil
import struct, subprocess, sys, threading, time, urllib2
import sqlite3, thread, urllib, dateparser
from datetime import timedelta

def exec_ipynb(filename_or_url):
    nb = (urllib2.urlopen(filename_or_url) if re.match(r'https?:', filename_or_url) else open(filename_or_url)).read()
    jsonNb = json.loads(nb)
    #check for the modified formatting of Jupyter Notebook v4
    if(jsonNb['nbformat'] == 4):
        exec '\n'.join([''.join(cell['source']) for cell in jsonNb['cells'] if cell['cell_type'] == 'code']) in globals()
    else:
        exec '\n'.join([''.join(cell['input']) for cell in jsonNb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']) in globals()

exec_ipynb('timelapse-utilities.ipynb')

In [29]:
exec_ipynb('votermap.ipynb')

In [156]:
exec_ipynb('geotools.ipynb')

In [18]:
load_voter_map(None)

Loading voter_map from voters/voter_map_05_09_13_17_18_18_18_l.pickle


In [23]:
len(voter_map)

1379040

In [46]:
all_vids=sorted(voter_map.keys())

In [20]:
load_vote_info(None)

Loading vote_map from voters/vote_info_map_05_09_13_17_18_18_18_a.pickle
Loading vote_arr from voters/vote_info_arr_05_09_13_17_18_18_18_a.pickle
Loading vote_index from voters/vote_info_index_05_09_13_17_18_18_18_a.pickle


In [30]:
load_block_id_use_map(None)

Loading block_id_use_map from voters/block_id_use_map_05_09_13_17_18_18_18_a.pickle


In [69]:
# Initialize pph_all_vids if it isn't already defined
try:
    pph_all_vids
except:
    phh_all_vids = set()

In [102]:
# Force reset of phh_all_vids
phh_all_vids = set()

In [98]:
# Use voter_map and block_id_use_map to find out what happened to 
# people living in closed public housing.
# Added Fairywood to include an area that seems to overlap Broadhead and Westgate
phh_map = {'EastMall':{'start':1969, 'end':2004, 'census_blocks':['420031115003008']},
           'Broadhead':{'start':1953, 'end':2004, 'census_blocks':['420035630001001','420035630001002','420035630001003','420035630001005','420035630001006','420035630001012','420035630001021','420035630001022']},
           'Westgate':{'start':1960, 'end':2002, 'census_blocks':['420035630001005','420035630001015','420035630001018']},
           'Fairywood':{'start':1960, 'end':2004,'census_blocks':['420035630001013']},
           'StClair':{'start':1949, 'end':2009,'census_blocks':['420035617001006','420035617001011','420035617001012','420035617001013']}
          }

In [99]:
for place in phh_map:
    voter_id_set=set()
    for census_block in phh_map[place]['census_blocks']:
        try:
            print "%s[%r]: %d voters total" % (place, census_block, block_id_use_map[census_block]['next'])
            voter_id_set = voter_id_set.union(set(filter(lambda x: x!='next' and x!='numpoints', block_id_use_map[census_block].keys())))
        except:
            print "%s[%r]: 0 voters total (Not in block_id_use_map)" % (place, census_block)
            
    phh_map[place]['voter_ids']=voter_id_set
    print "  %s Total: %d" %(place, len(phh_map[place]['voter_ids']))

Westgate['420035630001005']: 2 voters total
Westgate['420035630001015']: 37 voters total
Westgate['420035630001018']: 276 voters total
  Westgate Total: 315
EastMall['420031115003008']: 100 voters total
  EastMall Total: 100
Broadhead['420035630001001']: 4 voters total
Broadhead['420035630001002']: 84 voters total
Broadhead['420035630001003']: 2 voters total
Broadhead['420035630001005']: 2 voters total
Broadhead['420035630001006']: 2 voters total
Broadhead['420035630001012']: 2 voters total
Broadhead['420035630001021']: 42 voters total
Broadhead['420035630001022']: 44 voters total
  Broadhead Total: 181
StClair['420035617001006']: 35 voters total
StClair['420035617001011']: 252 voters total
StClair['420035617001012']: 139 voters total
StClair['420035617001013']: 81 voters total
  StClair Total: 489
Fairywood['420035630001013']: 699 voters total
  Fairywood Total: 699


In [100]:
len(phh_all_vids)

777

In [103]:
for place in phh_map:
    print place
    for vid in phh_map[place]['voter_ids']:
        vrec = voter_map[vid]
        for i in range(0,len(vrec['addr_arr'])):
            addr_rec = vrec['addr_arr'][i]
            if(addr_rec['census_block'] in phh_map[place]['census_blocks'] and
               addr_rec['date'].year<phh_map[place]['end']):
                print "  %r[%d of %d]: %r (cb=%r, date=%s)"%(vid, i, len(vrec['addr_arr']),addr_rec['address'],
                                                             addr_rec['census_block'], addr_rec['date'])
                phh_all_vids.add(vid)


Westgate
  '003017159-02'[0 of 2]: '190 VILLAGE RD PITTSBURGH PA 15205' (cb=u'420035630001015', date=1988-03-01 00:00:00)
  '002776840-02'[0 of 3]: '3640 NORTH CT PITTSBURGH PA 15205' (cb=u'420035630001005', date=1984-10-01 00:00:00)
  '002121235-02'[0 of 1]: '176 VILLAGE RD PITTSBURGH PA 15205' (cb=u'420035630001015', date=1998-07-01 00:00:00)
  '002935654-02'[0 of 4]: '490 VILLAGE RD PITTSBURGH PA 15205' (cb=u'420035630001018', date=1997-03-01 00:00:00)
  '002539985-02'[0 of 2]: '382 VILLAGE RD PITTSBURGH PA 15205' (cb=u'420035630001018', date=1984-09-01 00:00:00)
  '002406411-02'[0 of 1]: '150 VILLAGE RD PITTSBURGH PA 15205' (cb=u'420035630001015', date=1975-10-01 00:00:00)
  '004655810-02'[0 of 2]: '406 VILLAGE RD PITTSBURGH PA 15205' (cb=u'420035630001018', date=1990-07-03 00:00:00)
  '002765718-02'[0 of 4]: '3531 EAST CT PITTSBURGH PA 15205' (cb=u'420035630001005', date=1980-10-01 00:00:00)
  '013273808-02'[0 of 1]: '1904 VILLAGE RD PITTSBURGH PA 15205' (cb=u'420035630001018', da

In [157]:
PointToPixelXY(voter_map['001988007-02']['addr_arr'][0]['latlon'])

[71.10967254638672, 96.468017578125]

In [104]:
len(phh_all_vids)

717

In [105]:
phh_all_vids_file_path = 'voters/phh_all_vids_a.pickle'

# Save out canonical_addr_map
with open(phh_all_vids_file_path, 'wb') as handle:
    pickle.dump(phh_all_vids, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [106]:
num_move_map={}

In [108]:
for vid in all_vids:
    vrec = voter_map[vid]
    nmoves = len(vrec['addr_arr'])
    if not nmoves in num_move_map:
        num_move_map[nmoves]=set()
    num_move_map[nmoves].add(vid)


In [109]:
num_move_map.keys()

[1, 2, 3, 4, 5, 6, 7]

In [110]:
num_move_map[7]

{'002711635-02', '003069847-02'}

In [181]:
freq_move_vid_set=set()
freq_thresh = 4

In [182]:
for num in num_move_map.keys():
    print "%d: %d" % (num, len(num_move_map[num]))
    if num>=freq_thresh:
        freq_move_vid_set = freq_move_vid_set.union(num_move_map[num])

1: 995954
2: 270036
3: 87184
4: 23236
5: 2535
6: 93
7: 2


In [184]:
# freq_move_vids_a.pickle includes voters with 4 or more addresses
# freq_move_vids_6_b.pickle only includes voters with 6 or more addresses
freq_move_vids_file_path = 'voters/freq_move_vids_4_b.pickle'

# Save out canonical_addr_map
with open(freq_move_vids_file_path, 'wb') as handle:
    pickle.dump(freq_move_vid_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [177]:
len(freq_move_vid_set)

2630

In [136]:
def DatetimeEpochMonthStart(date):
    month_start_date = date.replace(day=1)
    return calendar.timegm(month_start_date.utctimetuple())

def DatetimeEpochYearStart(date):
    year_start_date = date.replace(day=1,month=1)
    return calendar.timegm(year_start_date.utctimetuple())

def add_one_month(t):
    """Return a `datetime.date` or `datetime.datetime` (as given) that is
    one month later.
    
    Note that the resultant day of the month might change if the following
    month has fewer days:
    
        >>> add_one_month(datetime.date(2010, 1, 31))
        datetime.date(2010, 2, 28)
    """
    one_day = timedelta(days=1)
    one_month_later = t + one_day
    while one_month_later.month == t.month:  # advance to start of next month
        one_month_later += one_day
    target_month = one_month_later.month
    while one_month_later.day < t.day:  # advance to appropriate day
        one_month_later += one_day
        if one_month_later.month != target_month:  # gone too far
            one_month_later -= one_day
            break
    return one_month_later

def add_one_year(t):
    """Return a `datetime.date` or `datetime.datetime` (as given) that is
    one year later.
    """
    one_year = timedelta(days=365)
    one_year_later = t + one_year
    return one_year_later

def DatetimeEpochNextMonth(date):
    month_start_date = date.replace(day=1)
    next_month_start_date = add_one_month(month_start_date)
    return calendar.timegm(next_month_start_date.utctimetuple())

def DatetimeEpochNextYear(date):
    year_start_date = date.replace(day=1,month=1)
    next_year_start_date = add_one_year(year_start_date)
    return calendar.timegm(next_year_start_date.utctimetuple())

In [185]:
# Adapted from http://localhost:8815/notebooks/uwsgi/dotmaptiles/server/gradeck/Process%20Gradeck%20Moves-FromGabriel-AW1.ipynb
# Monthly
import array, calendar, csv, json, math, random, re, time, datetime
import numpy as np

points = []

monthcount = {}

try:
    for vid in freq_move_vid_set:
        vrec = voter_map[vid]
        if len(vrec['addr_arr'])<2:
            # No moves, skip 
            continue
        # In case there are any missing 'latlon' in address records, which can happen when 
        # census_block=None, skip those moves
        for i in range(0,len(vrec['addr_arr'])-1):
            o_addr_rec = vrec['addr_arr'][i]
            d_addr_rec = vrec['addr_arr'][i+1]
            if(not 'latlon' in o_addr_rec or not 'latlon' in d_addr_rec):
                # Skip this one
                continue
            move_date = d_addr_rec['date']
            move_epoch = DatetimeEpochYearStart(move_date)
            # Always set arrival to be start of month after start of move
            arrival_epoch = DatetimeEpochNextYear(move_date)
            o_coords = o_addr_rec['latlon']
            d_coords = d_addr_rec['latlon']
            o_lat = float(o_coords.y)
            o_lng = float(o_coords.x)
            d_lat = float(d_coords.y)
            d_lng = float(d_coords.x)
            points += LngLatToPixelXY((o_lng, o_lat))
            points += LngLatToPixelXY(((o_lng+d_lng)*0.5, (o_lat+d_lat)*0.5))
            points += LngLatToPixelXY((d_lng, d_lat))
            points.append(move_epoch)
            points.append(arrival_epoch)
            #print "%s[%d]: [%d](%f,%f)->(%f,%f)[%d], len %d"%(vid,i,move_epoch,
            #                                                  o_lng,o_lat,
            #                                                  d_lng,d_lat,
            #                                                  arrival_epoch,len(points))
    
except Exception as e:
    print "Error processing vid %s: %s" % (vid, e)

freq_movers_path = 'voting/voter_freq_num_moves_05_09_13_17_18_18_18_{}_movers.bin'.format(freq_thresh)    
array.array('f', points).tofile(open(freq_movers_path, 'wb'))   
print "Wrote %d dots to %s"%(len(points)/8, freq_movers_path)

Wrote 80019 dots to voting/voter_freq_num_moves_05_09_13_17_18_18_18_4_movers.bin


In [179]:
voter_map['020170406-02']['addr_arr']

[{'address': '16 GILFORD CT PITTSBURGH PA 15206',
  'census_block': u'420031115004012',
  'date': datetime.datetime(2004, 7, 25, 0, 0),
  'latlon': <shapely.geometry.point.Point at 0x7f735d427990>},
 {'address': '918 MELLON ST PITTSBURGH PA 15206',
  'census_block': u'420031102004014',
  'date': Timestamp('2008-06-12 00:00:00'),
  'latlon': <shapely.geometry.point.Point at 0x7f735d427a50>},
 {'address': '5 BLANFORD CT PITTSBURGH PA 15206',
  'census_block': None,
  'date': Timestamp('2012-08-30 15:43:29')},
 {'address': '500 DELLA DR NORTH VERSAILLES PA 15137',
  'census_block': u'420035041002009',
  'date': Timestamp('2016-10-20 14:35:37'),
  'latlon': <shapely.geometry.point.Point at 0x7f735d427ad0>},
 {'address': '1153 PAULSON AVE PITTSBURGH PA 15206',
  'census_block': u'420031203001041',
  'date': Timestamp('2018-06-19 09:37:11'),
  'latlon': <shapely.geometry.point.Point at 0x7f735d427b50>}]

In [168]:
calendar.timegm(datetime.datetime(2000, 12, 1, 0, 0).utctimetuple())

975628800

In [160]:
PointToPixelXY((voter_map['001988007-02']['addr_arr'][0]['latlon']+voter_map['001988007-02']['addr_arr'][1]['latlon'])/2.0)

TypeError: unsupported operand type(s) for +: 'Point' and 'Point'